In [ ]:
import os
import numpy as np
import numpy.random as npr
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
#plt.switch_backend('agg')

from scipy.integrate import odeint

keras = tf.keras
tf.compat.v1.enable_eager_execution()

#from neural_ode import NeuralODE

import sys
current_dir = os.path.dirname(os.getcwd())
target_dir = os.path.sep.join(current_dir.split(os.path.sep)[:])
sys.path.append(os.path.join(os.path.dirname(target_dir),'../eABCSMC'))
sys.path.append(os.path.join(os.path.dirname(target_dir),'../Epidemics_response/Data'))
sys.path.append(target_dir)

%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

In [ ]:
def SIR(z, t, beta, gamma):

    S, I, R = z

    dS = - beta * I * S / N 
    dI = beta * I * S / N - gamma * I
    dR = gamma * I 

    dzdt = [dS, dI, dR]
    return dzdt

def SIR_diffparam(z, t, beta, beta2, gamma, gamma2, errato = 0):

    S, I, R = z

    dS = - beta * I * S / N 
    dI = beta2 * I * S / N - gamma * I
    dR = gamma2 * I 

    dzdt = [dS, dI, dR]
    return dzdt

# adattare il modello tempo-variabile -> così da considerare lockdown ecc? -> o con alcuni threshold

def plot_traj(trajectories, width = 1.):
    x1 = trajectories[:,0]
    x2 = trajectories[:,1]
    x3 = trajectories[:,2]
    s = plt.plot(x1, linewidth = width, label = 'Susceptible')
    i = plt.plot(x2, linewidth = width, label = 'Infected')
    r = plt.plot(x3, linewidth = width, label = 'Removed')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.title('Simulation with R0 = %f' %R0 )
    
    
def plot_traj_gray(trajectories, width = 1.):
    x1 = trajectories[:,0]
    x2 = trajectories[:,1]
    x3 = trajectories[:,2]
    s = plt.plot(x1, linewidth = width, color = 'lightgray')
    i = plt.plot(x2, linewidth = width, color = 'lightgray')
    r = plt.plot(x3, linewidth = width, color = 'lightgray')

In [ ]:
# oop con ereditarietà?

def SEIR(z, t, beta, gamma, lamb, mu, a):

    S, E, I, R = z

    dS = - beta * I * S / N 
    dE = beta * I * S / N - a * E
    dI = a * E - gamma  * I
    dR = gamma * I 

    dzdt = [dS, dE, dI, dR]
    return dzdt

def plot_traj_SEIR(trajectories, width = 1.):
    x1 = trajectories[:,0]
    x2 = trajectories[:,1]
    x3 = trajectories[:,2]
    x4 = trajectories[:,3]
    
    s = plt.plot(x1, linewidth = width, label = 'Susceptible')
    r = plt.plot(x2, linewidth = width, label = 'Exposed')
    i = plt.plot(x3, linewidth = width, label = 'Infected')
    r = plt.plot(x4, linewidth = width, label = 'Removed')
    
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.title('Simulation with R0 = %f' %(a*beta/gamma) )
    plt.show()

In [ ]:
data_size = 100

batch_time = 20  # tempo delle batches
niters = 500
#batch_size = 84  # dimensione delle batches
batch_size = data_size - batch_time - 1

N = 60000
infected_0 = 100
beta = 0.3 # farli time evolving?
gamma = 0.1

R0 = beta/gamma

In [ ]:
t_grid = np.linspace(0, data_size-1, data_size)  # uniformly spaced data? -> even though advantage is learning with not uniformly spaced data
z0 = [N - infected_0, infected_0, 0] # initial conditions
true_yy = odeint(SIR, z0, t_grid, args=(beta, gamma))  # potrebbe aver senso tenerli in memoria se è lento

In [ ]:
plot_traj(true_yy)

In [ ]:

true_y = true_yy.copy()



from stochastic.processes.continuous import FractionalBrownianMotion

fbm = FractionalBrownianMotion(hurst=0.2, t= data_size)
# in questo modo è autocorrelato negativamente -> giorno da tanti tamponi seguito da giorno da pochi tamponi

noise = abs(N/200 * fbm.sample(data_size-1))
noise_2 = abs(N/200 * fbm.sample(data_size-1))


true_y[:,0] = abs(true_y[:,0] - noise)        
true_y[:,1] = abs(true_y[:,1] + noise - noise_2)
true_y[:,2] = N - true_y[:,0] - true_y[:,1]

# problema in realtà è che i suscettibili e i rimossi possono crescere/decrescere




In [ ]:
#true_y = true_yy
plot_traj(true_y)

In [ ]:
sigma_s = np.std(true_y[:,0:1])  
sigma_i = np.std(true_y[:,1:2]) 
sigma_r = np.std(true_y[:,2:3]) 

true_y[:, 0:1] = true_y[:, 0:1]/sigma_s
true_y[:, 1:2] = true_y[:, 1:2]/sigma_i
true_y[:, 2:3] = true_y[:, 2:3]/sigma_r

In [ ]:
plot_traj(true_y)

In [ ]:
def get_batch():
    """Returns initial point and last point over sampled frament of trajectory"""
    starts = np.random.choice(np.arange(data_size - batch_time - 1, dtype=np.int64), batch_size, replace=False)
    # This randomly chooses from {0, 1, ... , data_size - batch_time - 1}, batch_size different elements
    batch_y0 = true_y[starts] 
    batch_yN = true_y[starts + batch_time]
    # The function returns a tensor composed by some y0 and the respective yN,
    # being y0 + DeltaT.
    return tf.cast(batch_y0, dtype=tf.float32), tf.cast(batch_yN, dtype=tf.float32)

In [ ]:
num_param = 5 # Number of parameters
para_num = num_param

t0 = t_grid[:batch_time][0]  # t0 = first element of t_grid
t1 = t_grid[:batch_time][-1]  # t1 = the element of t_grid at batch_time
t_in = np.linspace(t0, t1, 10)  # The time grid between t0 and t1

batch_y0, batch_yN = get_batch()  # Returns the first and the last y observed for each batch



In [ ]:
from neural_ode import NeuralODE

In [ ]:
batch_y0

In [ ]:
niters_pre = 500  # Number of iterations of the preconditioner

class ODEModel_pre(tf.keras.Model):
    def __init__(self):
        super(ODEModel_pre, self).__init__()
        self.Weights = tf.Variable(tf.random.normal([num_param, 1], dtype=tf.float32)*0.01, dtype=tf.float32)
    # Initializer: assign normally distributed random weights which are very close to zero

    def call(self, inputs, **kwargs):
        t, y = inputs
        h = y
        S = h[:, 0:1] * sigma_s
        I = h[:, 1:2] * sigma_i
        R = h[:, 2:3] * sigma_r
        

        # ma che sintassi di merda -> facciamolo adattativo

        p1 = self.Weights[0]
        p2 = self.Weights[1]
        p3 = self.Weights[2]
        p4 = self.Weights[3]
        p5 = self.Weights[4]

        dS = - p1 * I * S / N + p5 * R 
        dI = p2 * I * S / N - p3 * I 
        dR = p4 * I - p5 * R
        
        h_out = tf.concat([dS/sigma_s, dI/sigma_i, dR/sigma_r], 1)
        return h_out


model_pre = ODEModel_pre()  
neural_ode_pre = NeuralODE(model_pre, t_in) 
optimizer = tf.compat.v1.train.AdamOptimizer(3e-2)  

In [ ]:
@tf.function
def compute_gradients_and_update_pre(batch_y0, batch_yN):
    """Takes start positions (x0, y0) and final positions (xN, yN)"""
    pred_y = neural_ode_pre.forward(batch_y0)  # Predict y using Runge-Kutta 4 for each y0 in batch_y0
    with tf.GradientTape() as g_pre:
        g_pre.watch(pred_y)
        loss = tf.reduce_mean(input_tensor=(pred_y - batch_yN)**2) + tf.reduce_sum(input_tensor=tf.abs(model_pre.trainable_weights[0]))
        # This step is computing the loss function
    dLoss = g_pre.gradient(loss, pred_y)  # Here we compute the gradient of the loss function
    h_start, dfdh0, dWeights = neural_ode_pre.backward(pred_y, dLoss)  # Here we compute the dWeights
    optimizer.apply_gradients(zip(dWeights, model_pre.weights))  # Here we update the weights
    return loss, dWeights

#parameters_pre = np.zeros((para_num, niters_pre))
parameters_pre = np.zeros((para_num, 1))

for step in tqdm(range(niters_pre)):
    loss, dWeights = compute_gradients_and_update_pre(batch_y0, batch_yN)
    #parameters_pre[:,step] = np.reshape(model_pre.trainable_weights[0].numpy(),(5,))
    parameters_pre = model_pre.trainable_weights[0].numpy()
print(parameters_pre)

In [ ]:
initial_weight = parameters_pre  # We initialize the weights with the parameters found in preconditioning
print(initial_weight.shape, "here")


class ODEModel(tf.keras.Model):
    def __init__(self):
        super(ODEModel, self).__init__()
        self.Weights = tf.Variable(tf.random.normal([num_param, 1], dtype=tf.float32)*0.01, dtype=tf.float32)
        # Initializer, initializes the weight to normal random variables with sd = 0.01

    def call(self, inputs, **kwargs):
        t, y = inputs
        h = y
        S = h[:, 0:1] * sigma_s
        I = h[:, 1:2] * sigma_i
        R = h[:, 2:3] * sigma_r
        

        # ma che sintassi di merda -> facciamolo adattativo

        p1 = self.Weights[0]
        p2 = self.Weights[1]
        p3 = self.Weights[2]
        p4 = self.Weights[3]
        p5 = self.Weights[4]

        dS = - p1 * I * S / N + p5 * R 
        dI = p2 * I * S / N - p3 * I 
        dR = p4 * I - p5 * R
        
        h_out = tf.concat([dS/sigma_s, dI/sigma_i, dR/sigma_r], 1)
        return h_out


model = ODEModel()
neural_ode = NeuralODE(model, t=t_in)  # We assign to NeuralODE the just created model and the time grid  between t0 and t1

@tf.function
def compute_gradients_and_update(batch_y0, batch_yN): 
    """Takes start positions (x0, y0) and final positions (xN, yN)"""
    pred_y = neural_ode.forward(batch_y0)  # This finds the predicted yNs
    with tf.GradientTape() as g:
        g.watch(pred_y)
        loss = tf.reduce_sum(input_tensor=(pred_y - batch_yN)**2)  # This creates the loss function

    dLoss = g.gradient(loss, pred_y)  # This computes the gradient of the loss function
    h_start, dfdh0, dWeights = neural_ode.backward(pred_y, dLoss)  # This applies the gradient descent to find
    # the updates for the weights

    return loss, dWeights

In [ ]:
def kinetic_energy(V, loggamma_v, loglambda_v):
    q = (np.sum(-V**2)/mom_theta - loggamma_v**2/mom_gamma - loglambda_v**2/mom_lambda)/2.0
    return q


def compute_gradient_param(dWeights, loggamma, loglambda, batch_size, para_num):
    WW = model.trainable_weights[0].numpy()
    dWeights = np.exp(loggamma)/2.0 * dWeights + np.exp(loglambda) * np.sign(WW - w_means)
    return dWeights


def compute_gradient_hyper(loss, weights, loggamma, loglambda, batch_size, para_num):
    grad_loggamma = np.exp(loggamma) * (loss/2.0 + 1.0) - (batch_size/2.0 + 1.0)
    grad_loglambda = np.exp(loglambda) * (np.sum(np.abs(weights - w_means)) + 1.0) - (para_num + 1.0)
    # This somehow computes the gradient of the hyper parameters in order to update them from step to step

    return grad_loggamma, grad_loglambda


def compute_Hamiltonian(loss, weights, loggamma, loglambda, batch_size, para_num):
    H = np.exp(loggamma)*(loss/2.0 + 1.0) + np.exp(loglambda)*(np.sum(np.abs(weights - w_means)) + 1.0) \
             - (batch_size/2.0 + 1.0) * loggamma - (para_num + 1.0) * loglambda
    return H

def leap_frog(v_in, w_in, loggamma_in, loglambda_in, loggamma_v_in, loglambda_v_in):
    # Leap frog step for Hamiltonian Monte Carlo

    model.trainable_weights[0].assign(w_in)
    v_new = v_in
    loggamma_v_new = loggamma_v_in
    loglambda_v_new = loglambda_v_in

    loggamma_new = loggamma_in
    loglambda_new = loglambda_in
    w_new = w_in

    for m in range(L):
        loss, dWeights = compute_gradients_and_update(batch_y0, batch_yN) # evaluate the gradient

        dWeights = np.asarray(dWeights[0])  # make the gradient to be numpy array
        dWeights = compute_gradient_param(dWeights, loggamma_new, loglambda_new, batch_size, para_num)
        grad_loggamma, grad_loglambda = compute_gradient_hyper(loss, w_new, loggamma_new, loglambda_new, batch_size, para_num)

        loggamma_v_new = loggamma_v_new - epsilon/2*grad_loggamma  # Computation of new velocities
        loglambda_v_new = loglambda_v_new - epsilon/2*grad_loglambda  # 2nd component of velocity
        v_new = v_new - epsilon/2*(dWeights)  # 3rd component of velocity
        w_new = model.trainable_weights[0].numpy() + epsilon / mom_theta * v_new  # updating of x_1
        model.trainable_weights[0].assign(w_new)  # Assignment
        loggamma_new = loggamma_new + epsilon / mom_gamma * loggamma_v_new  # Updating of x_2
        loglambda_new = loglambda_new + epsilon / mom_lambda * loglambda_v_new  # Updating of x_3

        # Second half of the leap frog
        loss, dWeights = compute_gradients_and_update(batch_y0, batch_yN)
        dWeights = np.asarray(dWeights[0])
        dWeights = compute_gradient_param(dWeights, loggamma_new, loglambda_new, batch_size, para_num)
        grad_loggamma, grad_loglambda = compute_gradient_hyper(loss, w_new, loggamma_new, loglambda_new, batch_size, para_num)

        v_new = v_new - epsilon/2*(dWeights)  # Updating of new velocities
        loggamma_v_new = loggamma_v_new - epsilon/2*grad_loggamma  # Updating of new velocities
        loglambda_v_new = loglambda_v_new - epsilon/2*grad_loglambda  # Updating of new velocities

    return v_new, w_new, loggamma_new, loglambda_new, loggamma_v_new, loglambda_v_new


neural_ode_test = NeuralODE(model, t=t_grid[0:data_size:20])
parameters = np.zeros((niters, para_num))  # book keeping the parameters
loggammalist = np.zeros((niters, 1))  # book keeping the loggamma
loglambdalist = np.zeros((niters, 1))  # book keeping the loggamma
loglikelihood = np.zeros((niters, 1))  # book keeping the loggamma
mom_theta = 0.2 # Momentum for the Hamiltonian Monte Carlo for theta parameters
mom_lambda = 0.1  # Momentum for the Hamiltonian Monte Carlo for lambda parameters
mom_gamma = 0.1  # Momentum for the Hamiltonian Monte Carlo for gamma parameters
L = 30  # leap frog step number
epsilon = 0.001  # leap frog step size  0.001
epsilon_max = 0.0002    # max 0.001  0.0002
epsilon_min = 0.0002    # max 0.001  0.0002
acc_rate = 0  # Used to compute and display the acceptance rate of the candidates
w_means = np.array([0, 0, 0, 0, 0], dtype=np.float32)  # A priori means of parameters of the model
w_means = w_means.reshape(5, 1)

def compute_epsilon(step):
    # This will compute the epsilon to use in the leapfrog which is different for every step. It decreases
    # with the steps increasing in number
    coefficient = np.log(epsilon_max/epsilon_min)
    return epsilon_max * np.exp(- step * coefficient / niters)


# initial weight
w_temp = initial_weight  # The one we found from the preconditioner
print("initial_w", w_temp)
loggamma_temp = 4. + np.random.normal()
loglambda_temp = np.random.normal()

model.trainable_weights[0].assign(w_temp)  # We assign to the weights of the model, the ones we found through
# the pre conditioner. Remember that the initial weights set by the initializer were random distributed according
# to a Gaussian with 0 mean and 0.01 sd.
loss_original, _ = compute_gradients_and_update(batch_y0, batch_yN)  # Compute the initial Hamiltonian

loggamma_temp = np.log(batch_size / loss_original)  # We define an initial guess for loggamma ?? Why defined as such?

print("This is the initial guess of log(gamma)", loggamma_temp, "with loss", loss_original)
if loggamma_temp > 6.:
    loggamma_temp = 6.
    epsilon_max = 0.0002
    epsilon_min = 0.0002

# training steps
for step in tqdm(range(niters)):

    epsilon = compute_epsilon(step)  # Compute the adaptive epsilon for the steps
    v_initial = np.sqrt(mom_theta)*np.random.randn(para_num, 1)  # initialize the velocity
    loggamma_v_initial = np.sqrt(mom_gamma)*np.random.normal()
    loglambda_v_initial = np.sqrt(mom_lambda)*np.random.normal()

    loss_initial, _ = compute_gradients_and_update(batch_y0, batch_yN)  # compute the initial Hamiltonian
    # This line uses the weights of the preconditioner (see line 303) to compute the loss function with those
    loss_initial = compute_Hamiltonian(loss_initial, w_temp, loggamma_temp, loglambda_temp, batch_size, para_num)
    # Then it computes the Hamiltonian

    v_new, w_new, loggamma_new, loglambda_new, loggamma_v_new, loglambda_v_new = \
                            leap_frog(v_initial, w_temp, loggamma_temp, loglambda_temp, loggamma_v_initial, loglambda_v_initial)

    # Then the leapfrog is applied in order to update the parameters and the hyper parameters and to further
    # optimize the weights estimation (L steps of leapfrog at a time)

    # compute the final Hamiltonian, using the updated sampled weights found through leapfrog
    loss_finial, _ = compute_gradients_and_update(batch_y0, batch_yN)
    loss_finial = compute_Hamiltonian(loss_finial, w_new, loggamma_new, loglambda_new, batch_size, para_num)

    # making decisions
    p_temp = np.exp(-loss_finial + loss_initial + \
                    kinetic_energy(v_new, loggamma_v_new, loglambda_v_new) - kinetic_energy(v_initial, loggamma_v_initial, loglambda_v_initial))

    p = min(1, p_temp)
    p_decision = np.random.uniform()
    if p > p_decision:
        parameters[step:step+1, :] = np.transpose(w_new)  # Parameters are updated
        w_temp = w_new
        loggammalist[step, 0] = loggamma_new
        loglambdalist[step, 0] = loglambda_new
        loglikelihood[step, 0] = loss_finial
        loggamma_temp = loggamma_new
        loglambda_temp = loglambda_new
        acc_rate += 1
    else:
        parameters[step:step+1, :] = np.transpose(w_temp)  # New parameters are not updated
        model.trainable_weights[0].assign(w_temp)
        loggammalist[step, 0] = loggamma_temp
        loglambdalist[step, 0] = loglambda_temp
        loglikelihood[step, 0] = loss_initial

    #print('Probability of acceptance: ', p)
    #print('Accepted: ', p > p_decision)
    print(parameters[step:step+1, :])

print('Acceptance rate: ', acc_rate/niters)

np.save('parameters', parameters)  # The Monte Carlo chain of the parameters
np.save('loggammalist', loggammalist)  # The Monte Carlo chain of loggamma
np.save('loglikelihood', loglikelihood)  # The Monte Carlo chain of losses


In [ ]:
plt.plot(parameters)

In [ ]:
from statsmodels.graphics import tsaplots
tsaplots.plot_acf(parameters[:,4])

In [ ]:
plt.plot(loglambdalist)
plt.plot(loggammalist)

In [ ]:
import seaborn as sns
import pandas as pd

g = sns.PairGrid(pd.DataFrame(parameters))
g.map_upper(sns.histplot)
g.map_lower(sns.kdeplot, fill=True)
g.map_diag(sns.histplot, kde=True)

In [ ]:
t_grid = np.linspace(0, data_size, data_size)  # uniformly spaced data? -> even though advantage is learning with not uniformly spaced data
z0 = [N - infected_0, infected_0, 0] # initial conditions

for j in tqdm(range(parameters.shape[0])):
    #simul_yy =  odeint(SIR_diffparam, z0, t_grid, args=tuple([np.median(parameters[:,i]) for i in range(parameters.shape[1])]))
    simul_yy =  odeint(SIR_diffparam, z0, t_grid, args=tuple(parameters[j]))
    plot_traj_gray(simul_yy)

true_yy = odeint(SIR, z0, t_grid, args=(beta, gamma))  # potrebbe aver senso tenerli in memoria se è lento
plot_traj(true_yy)

    
plt.show()

In [ ]:
from scipy.stats import wilcoxon, mannwhitneyu, kruskal

kruskal(parameters[:,0],parameters[:,1])

In [ ]:
import arviz as az

az.ess(parameters[:,3])

In [ ]:
az.plot_posterior(parameters[:,2])

In [ ]:
az.hdi(parameters[:,3])

In [ ]:
az.geweke(parameters[:,1])

In [ ]:
az.plot_mcse(parameters[:,0])

In [ ]:
# TO DO

# - simulation with only two parameters
# - go back to noisy simulation
# - thinning
# - weighted noise
# - CI for intensive therapy
# - sensitivity analysis!

# - understand why bayesian
# - sliding window for parameters estimation?
# - non observable states' estimation (through t-1?)
# - different dynamical systems interactions 

# - batch does not really make sense
# -> we always overestimate